<a href="https://colab.research.google.com/github/Erny-BR/notasRelevantes/blob/main/monitoramentoTreinamentoPyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ao definir a arquitetura de uma rede CNN com a classe `torch.nn` no PyTorch, você pode usar vários recursos para acompanhar o fluxo de inputs e pesos (weights) durante o treinamento. Isso é crucial para depurar, entender o comportamento da rede e otimizar seu desempenho. Aqui estão algumas técnicas e ferramentas úteis:

**1. Hooks (Ganchos):**

*   **O que são:** Hooks são funções que podem ser registradas em `nn.Module`s ou `Tensor`s. Eles são chamados durante o passo `forward` (hook de forward) ou `backward` (hook de backward).
*   **Como usar:**
    *   **Hooks de Forward:** Permitem inspecionar ou modificar as entradas ou saídas de uma camada durante o passo forward.
    *   **Hooks de Backward:** Permitem inspecionar ou modificar os gradientes durante a propagação do gradiente (backpropagation).
*   **Exemplo (inspecionando a saída de uma camada):**

In [ ]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        return x

model = MyModel()

# Função hook para imprimir a saída da camada conv1
def print_output_hook(module, input, output):
    print(f"Saída da camada {module.__class__.__name__}:")
    print(output)

# Registrar o hook na camada conv1
hook_handle = model.conv1.register_forward_hook(print_output_hook)

# Realizar um passo forward
input = torch.randn(1, 3, 28, 28)
output = model(input)

# Remover o hook quando não for mais necessário
hook_handle.remove()

**2.  `torch.nn.Module.named_parameters()` e `torch.nn.Module.named_buffers()`:**

*   **O que são:** Métodos que permitem iterar sobre os parâmetros e buffers (como médias e variâncias de batch normalization) de um modelo, juntamente com seus nomes.
*   **Como usar:** Útil para imprimir os valores dos pesos, gradientes e outros tensores registrados como parâmetros ou buffers.
*   **Exemplo (imprimindo pesos e gradientes):**

In [ ]:
for name, param in model.named_parameters():
    print(f"Camada: {name} | Tamanho: {param.size()} | Valores : {param[:2]} \n Gradiente: {param.grad}")

**3.  `torch.autograd.grad_check()` (para verificação de gradientes):**

*   **O que é:** Uma função utilitária que ajuda a verificar numericamente a correção dos gradientes calculados durante a backpropagation.
*   **Como usar:** Compara os gradientes analíticos calculados pelo PyTorch com estimativas numéricas de gradientes. Útil para depurar implementações customizadas de camadas ou funções de perda.

**4.  Visualização com TensorBoard (ou Visdom):**

*   **O que é:** O TensorBoard (originalmente do TensorFlow, mas suportado pelo PyTorch) é uma ferramenta de visualização para monitorar o treinamento de redes neurais. O Visdom é uma alternativa mais leve e originalmente criada para PyTorch.
*   **Como usar:**
    *   **`torch.utils.tensorboard.SummaryWriter`:** Permite registrar escalares (como perda e acurácia), histogramas (distribuição de pesos e ativações), imagens (como mapas de ativação) e o grafo da rede.
    *   **Visualizar no navegador:** O TensorBoard (ou Visdom) exibe essas informações em um painel interativo em seu navegador, permitindo que você acompanhe o progresso do treinamento e visualize o comportamento interno da rede.

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# Criar um SummaryWriter
writer = SummaryWriter('runs/my_experiment')

# ... dentro do loop de treinamento ...

# Registrar a perda
writer.add_scalar('Loss/train', loss.item(), epoch)

# Registrar um histograma dos pesos da camada conv1
writer.add_histogram('Conv1/weights', model.conv1.weight, epoch)

# Fechar o writer quando terminar
writer.close()

# Para visualizar:
# No terminal, execute: tensorboard --logdir runs
# Abra o navegador em: http://localhost:6006/

**5. Impressão intermitente (com instruções `print`):**

*   **O que é:** A maneira mais simples de inspecionar valores é usar instruções `print` dentro do método `forward` da sua classe `nn.Module`.
*   **Como usar:** Pode ser útil para verificações rápidas, mas pode gerar muita saída para redes grandes ou durante o treinamento de muitas épocas.

**Dicas:**

*   **Seja seletivo:** Não tente inspecionar tudo ao mesmo tempo. Concentre-se nas camadas ou tensores mais relevantes para o problema que você está investigando.
*   **Use hooks com cuidado:** Hooks podem adicionar sobrecarga computacional, especialmente se forem chamados com frequência. Remova-os quando não forem mais necessários.
*   **Combine técnicas:** Use uma combinação dessas técnicas para obter uma compreensão completa do fluxo de dados e pesos em sua rede.

Ao usar essas ferramentas e técnicas, você pode obter insights valiosos sobre o funcionamento interno de suas redes CNN no PyTorch, ajudando a depurar, otimizar e melhorar seus modelos de aprendizado profundo. Lembre-se de que a escolha das ferramentas dependerá das necessidades específicas do seu projeto e do que você deseja investigar.